In [27]:
%matplotlib inline
import sys

from pathlib import Path
from datetime import timedelta
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,f1_score

import dateutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
import sklearn
from keras.callbacks import EarlyStopping
from tensorflow.keras import Model ,models, layers, optimizers, utils

from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from google.colab import drive
drive.mount('/content/drive')
!pip install ./drive/MyDrive/ICS_anomaly_detection/eTaPR-21.8.2-py3-none-any.whl
from TaPR_pkg import etapr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./drive/MyDrive/ICS_anomaly_detection/eTaPR-21.8.2-py3-none-any.whl
eTaPR is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [28]:
attack_answer = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/attack_answer.csv", header=None, names=['num','attack'])
attack_answer = attack_answer[['attack']]
attack_answer = attack_answer.drop(index=0, axis=-0)
pred_1 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/result_1.csv", header=None, names=['num','target1'])
pred_1 = pred_1[['target1']]
pred_1 = pred_1.drop(index=0, axis=0)
pred_2 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/result_2.csv", header=None, names=['num','target1'])
pred_2 = pred_2[['target1']]
pred_2 = pred_2.drop(index=0, axis=0)
pred_3 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/result_3.csv", header=None, names=['num','target1'])
pred_3 = pred_3[['target1']]
pred_3 = pred_3.drop(index=0, axis=0)
pred_4 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/result_4.csv", header=None, names=['num','target1'])
pred_4 = pred_4[['target1']]
pred_4 = pred_4.drop(index=0, axis=0)
pred_5 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/group3/result_1.csv", header=None, names=['num','target1'])
pred_5 = pred_5[['target1']]
pred_5 = pred_5.drop(index=0, axis=0)
pred_6 = pd.read_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/result_6.csv", header=None, names=['num','target1'])
pred_6 = pred_6[['target1']]
pred_6 = pred_6.drop(index=0, axis=0)
result = pd.concat([pred_1,pred_2,pred_3,pred_4,pred_5,pred_6], axis=1)

In [29]:
attack_answer = np.where(attack_answer == 1,1,0)
np.unique(attack_answer,return_counts=True)
result = result.astype(int)
attack_answer = attack_answer.reshape(-1)
result = result.sum(axis=1)
result.reset_index(drop=True,inplace=True)

In [30]:
pred_attack = np.zeros(result.shape[0])
for i in range(result.shape[0]):
  if result[i] == 0:
    pred_attack[i]=0
  else:
    pred_attack[i]=1



In [31]:
print(len(pred_attack))
print(len(pred_attack[pred_attack==1]))

336600
10594


In [32]:
print(confusion_matrix(attack_answer,pred_attack))
print(f1_score(attack_answer, pred_attack))
TaPR = etapr.evaluate_haicon(anomalies=attack_answer, predictions=pred_attack)
print(f"F1: {TaPR['f1']:.3f} (TaP: {TaPR['TaP']:.3f}, TaR: {TaPR['TaR']:.3f})")
print(f"# of detected anomalies: {len(TaPR['Detected_Anomalies'])}")

[[322046   3702]
 [  3960   6892]]
0.6427305791289751
F1: 0.523 (TaP: 0.476, TaR: 0.582)
# of detected anomalies: 37


In [33]:
pd.DataFrame(pred_attack).to_csv("./drive/MyDrive/ICS_anomaly_detection/hai-22.04/results/final.csv")